In [1]:
import smartsheet
import os

SMARTSHEET_ACCESS_TOKEN = "74xq6QRJHXtN4VdRbwCl9rJeznxjgBcKEPJeJ" 

try:
    smartsheet_client = smartsheet.Smartsheet(SMARTSHEET_ACCESS_TOKEN)


except Exception as e:
    print(f"Details: {e}")
    exit()

def list_all_sheets():
    print("--- Connecting to Smartsheet API ---")
    
    try:
        response = smartsheet_client.Sheets.list_sheets(include_all=True)
        
        sheets = response.data
        
        if not sheets:
            print("No sheets found in your account.")
            return

        print(f"Found {len(sheets)} sheets.")
        print("-" * 30)

        for sheet in sheets:
            print(f"ID: {sheet.id} | Name: {sheet.name} | Owner: {sheet.owner}")

        print("-" * 30)
        
        # We can use the ID of the first sheet found for the next step.
        return sheets[0].id

    except smartsheet.exceptions.ApiError as e:
        print(f"\nAPI Error encountered: {e.error.message}")
        print("Please check that your access token has the necessary permissions (scope: sheets:read).")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def get_sheet_data(sheet_id):
    """
    Retrieves the full row and cell data for a specific sheet ID.
    """
    if not sheet_id:
        return
        
    print(f"\n--- Retrieving Full Data for Sheet ID: {sheet_id} ---")

    try:
        # Retrieve the entire sheet object, including rows, columns, and cell data.
        sheet_object = smartsheet_client.Sheets.get_sheet(sheet_id)

        print(f"Successfully loaded sheet: {sheet_object.name}")
        
        # 1. Get Column Mapping (needed to translate column IDs to names)
        column_map = {col.id: col.title for col in sheet_object.columns}
        print(f"Columns: {list(column_map.values())}")
        
        print("\nFirst 5 Rows of Data:")
        print("-" * 30)
        
        # 2. Process Row Data
        for i, row in enumerate(sheet_object.rows):
            if i >= 5: # Limit output to 5 rows for brevity
                break
            
            row_data = {}
            for cell in row.cells:
                # Use the column map to find the column name
                column_title = column_map.get(cell.column_id, "Unknown Column")
                
                # Check for a display value first, then fallback to the raw value
                value = cell.display_value if cell.display_value else cell.value
                
                # Store data: {Column Name: Cell Value}
                row_data[column_title] = value
                
            print(f"Row {row.row_number}: {row_data}")
        print("-" * 30)

    except smartsheet.exceptions.ApiError as e:
        print(f"\nAPI Error encountered: {e.error.message}")
        print("Please ensure your token has sheets:read scope and the sheet ID is correct.")
    except Exception as e:
        print(f"An unexpected error occurred during data retrieval: {e}")


# --- EXECUTION ---
if __name__ == "__main__":
    # Step 1: List all sheets to verify connection and get a target ID
    first_sheet_id = list_all_sheets()
    
    # Step 2: Retrieve detailed data from the first sheet found
    if first_sheet_id:
        get_sheet_data(first_sheet_id)
    
    print("\nScript finished.")
    


--- Connecting to Smartsheet API ---
Found 25 sheets.
------------------------------
ID: 595984821211012 | Name: Bainbridge Permit Tracking Sheet | Owner: None
ID: 3995192664215428 | Name: Bella Vita Farm Permit Tracking Sheet | Owner: None
ID: 434783269113732 | Name: Burnt Hill Farm Permit Tracking Sheet | Owner: None
ID: 4936707175567236 | Name: Communication Log - Bainbridge | Owner: None
ID: 4726527783554948 | Name: Communication Log - Bella Vita Farm | Owner: None
ID: 2819825047916420 | Name: Communication Log - Burnt Hill Farm | Owner: None
ID: 110397865545604 | Name: Communication Log - Loch Raven | Owner: None
ID: 1457022584180612 | Name: Communication Log - Pimlico | Owner: None
ID: 3339646336454532 | Name: Communication Log - Quantum | Owner: None
ID: 1069788332773252 | Name: Communication Log - SBY Market | Owner: None
ID: 8310218912911236 | Name: DNR - Pilot Project Questionnaire | Owner: None
ID: 2137064200949636 | Name: GEC_Permitting_Inventory_2024 | Owner: None
ID: 8555